In [20]:
import pandas as pd
import datetime as dt
from helpers import *

In [3]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name='Year 2010-2011')

In [25]:
df = df_.copy()

In [26]:
check_df(df)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                 WHITE METAL LANTERN         6   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom  
##################### NA #####################
Invoice             0

In [27]:
# Drop NA's
df.dropna(axis=0, inplace=True)
# C means refundees, drop them
df = df[~df["Invoice"].str.contains("C", na=False)]
# there shouldn't be quantity value as zero
df = df[df["Quantity"] > 0]

In [29]:
# Replace outliers with threshold (0.99-0.01)
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

# RFM Segmentation

In [35]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [36]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [37]:
today_date = dt.datetime(2011, 12, 11)

In [40]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda price: price.sum()})

In [41]:
rfm.columns = ['recency', 'frequency', "monetary"]

In [42]:
rfm = rfm[(rfm['monetary'] > 0)]

# RFM Scores

In [43]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

# RFM Segmentation

In [45]:
rfm['rfm_segment'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

In [46]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [48]:
rfm['rfm_segment'] = rfm['rfm_segment'].replace(seg_map, regex=True)
rfm = rfm[["recency", "frequency", "monetary", "rfm_segment"]]

In [49]:
rfm.head()

,recency,frequency,monetary,rfm_segment
Customer ID,,,,
12346.0,326,1,310.44,hibernating
12347.0,3,7,4310.00,champions
12348.0,76,4,1770.78,at_risk
12349.0,19,1,1491.72,promising
12350.0,311,1,331.46,hibernating
